# Packages Installs and environment preparation

In [7]:
# !7z x data/20200425_UOWM_IEC104_Dataset_m_sp_na_1_DoS.7z
# !7z x data/20200426_UOWM_IEC104_Dataset_c_ci_na_1.7z
# !7z x data/20200426_UOWM_IEC104_Dataset_c_ci_na_1_DoS.7z
# !7z x data/20200427_UOWM_IEC104_Dataset_c_se_na_1.7z
# !7z x data/20200428_UOWM_IEC104_Dataset_c_sc_na_1.7z
# !7z x data/20200428_UOWM_IEC104_Dataset_c_se_na_1_DoS.7z
# !7z x data/20200429_UOWM_IEC104_Dataset_c_sc_na_1_DoS.7z
# !7z x data/20200605_UOWM_IEC104_Dataset_c_rd_na_1.7z
# !7z x data/20200608_UOWM_IEC104_Dataset_mitm_drop.7z
# !7z x data/20200606_UOWM_IEC104_Dataset_c_rp_na_1_DoS.7z
# !7z x data/20200605_UOWM_IEC104_Dataset_c_rd_na_1_DoS.7z
# !7z x data/20200606_UOWM_IEC104_Dataset_c_rp_na_1.7z

# Imports

In [2]:
from scapy.all import *
import pandas as pd
# from catboost import CatBoostClassifier
# import shap
import seaborn as sns
# import feature_engine
# import feature_engine.encoding
import numpy as np
# from catboost import Pool
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from collections import defaultdict
import os
# from google.colab import drive
# drive.mount('/content/drive')
TIMEOUT = 30

# Code

In [9]:
def get_attack_directory_attack(path):
  return path.split('Dataset_')[1]
attacks_directories= [
                      '20200425_UOWM_IEC104_Dataset_m_sp_na_1_DoS',
                      '20200426_UOWM_IEC104_Dataset_c_ci_na_1',
                      '20200427_UOWM_IEC104_Dataset_c_se_na_1',
                      '20200428_UOWM_IEC104_Dataset_c_sc_na_1',
                      '20200605_UOWM_IEC104_Dataset_c_rd_na_1',
                      '20200606_UOWM_IEC104_Dataset_c_rp_na_1',
                      '20200426_UOWM_IEC104_Dataset_c_ci_na_1_DoS',
                      '20200605_UOWM_IEC104_Dataset_c_rd_na_1_DoS',
                      '20200606_UOWM_IEC104_Dataset_c_rp_na_1_DoS',
                      '20200428_UOWM_IEC104_Dataset_c_se_na_1_DoS',
                      '20200429_UOWM_IEC104_Dataset_c_sc_na_1_DoS',
                                            # '20200608_UOWM_IEC104_Dataset_mitm_drop'
                                            ]
devices_to_read = ['qtester','attacker1','attacker2','attacker3']

# if '20200425_UOWM_IEC104_Dataset_m_sp_na_1_DoS':
#   not
get_attack_directory_attack(attacks_directories[0])

'm_sp_na_1_DoS'

In [10]:
a={}
for attack_directory in attacks_directories:
  for device_to_read in devices_to_read:
    pcap_file = f"data/{attack_directory}/{attack_directory}_{device_to_read}/{attack_directory}_{device_to_read}_iec104_only.pcap"
    if device_to_read != 'qtester' and attack_directory == '20200425_UOWM_IEC104_Dataset_m_sp_na_1_DoS':
      continue
    packets=rdpcap(pcap_file,count=200)
    for packet in packets:
      a[len(packet[TCP].payload)] = 1
MAX_IEC104_PAYLOAD_BYTES = max(a)

In [11]:
MAX_IEC104_PAYLOAD_BYTES

88

In [12]:
from pathlib import Path
all_packets = []
for attack_directory in attacks_directories:
#   dfs=[]
  print(attack_directory)
  for device_to_read in devices_to_read:
    if device_to_read != 'qtester' and attack_directory == '20200425_UOWM_IEC104_Dataset_m_sp_na_1_DoS':
      continue
    pcap_file = f"./data/{attack_directory}/{attack_directory}_{device_to_read}/{attack_directory}_{device_to_read}_iec104_only.pcap"
    # print(device_to_read,attack_directory)
    # df = pd.read_csv(f'/content/{os.path.basename(pcap_file)}.npt', index_col=0)
    packets=rdpcap(pcap_file)
    if device_to_read != 'qtester':
      label = get_attack_directory_attack(attack_directory)
    else:
      label = 'normal'
    if device_to_read == 'qtester' and attack_directory == '20200425_UOWM_IEC104_Dataset_m_sp_na_1_DoS':
      label = get_attack_directory_attack(attack_directory)
      
    all_packets.append([packets,label,get_attack_directory_attack(attack_directory)])
    # df['label'] = label
    # print(df.columns)
    # print(df.index)
    # df['flow'] =  df['flow']+'_'+df['label']
    # df.index = df.index.map(lambda x: x +'_'+ get_attack_directory_attack(attack_directory))
    # df['flow']=df['flow']
    # print(df.head())
    # dfs.append(df)
#   df = pd.concat(dfs)
#   df.to_parquet(f"./data/iec104v2_15.parquet")

20200425_UOWM_IEC104_Dataset_m_sp_na_1_DoS
20200426_UOWM_IEC104_Dataset_c_ci_na_1
20200427_UOWM_IEC104_Dataset_c_se_na_1
20200428_UOWM_IEC104_Dataset_c_sc_na_1
20200605_UOWM_IEC104_Dataset_c_rd_na_1
20200606_UOWM_IEC104_Dataset_c_rp_na_1
20200426_UOWM_IEC104_Dataset_c_ci_na_1_DoS
20200605_UOWM_IEC104_Dataset_c_rd_na_1_DoS
20200606_UOWM_IEC104_Dataset_c_rp_na_1_DoS
20200428_UOWM_IEC104_Dataset_c_se_na_1_DoS
20200429_UOWM_IEC104_Dataset_c_sc_na_1_DoS


In [13]:
def get_sessions_with_timeout(timeout,experiment_attack_directory):
    last_packet_time = defaultdict(lambda: 0)  # Initialize last_packet_time dictionary with default value 0
    flow_bin = defaultdict(lambda: 0)
    def session_extractor(packet):
        sess = [experiment_attack_directory,"TCP", packet["IP"].src, packet["TCP"].sport,
                                   packet["IP"].dst, packet["TCP"].dport]

        sess_key = str(sorted(sess, key=str))
        current_time = packet.time

        # Track original session key time
        if current_time - last_packet_time[sess_key] > timeout:
            flow_bin[sess_key]+=1
            # If the timeout is exceeded, modify the session key to create a new session
        sess.append(flow_bin[sess_key])
        last_packet_time[sess_key] = current_time
        sess_key = str(sorted(sess, key=str))
        return sess_key
    return session_extractor

In [31]:
def build_dataframe(packet_list):
    ip_fields = ['ip_' + field.name for field in IP().fields_desc]
    tcp_fields = ['tcp_' + field.name for field in TCP().fields_desc]
    ip_fields_raw = [field.name for field in IP().fields_desc]
    tcp_fields_raw = [field.name for field in TCP().fields_desc]

    dataframe_fields = ip_fields + ['time'] + tcp_fields + ['size', 'payload', 'payload_raw']

    rows = []
    for packet in packet_list[IP]:
        field_values = []
        for field in ip_fields_raw:
            if field == 'options':
                field_values.append(len(packet[IP].fields[field]))
            else:
                field_values.append(packet[IP].fields[field])

        field_values.append(packet.time)

        layer_type = type(packet[IP].payload)
        for field in tcp_fields_raw:
            try:
                if field == 'options':
                    field_values.append(len(packet[layer_type].fields[field]))
                else:
                    field_values.append(packet[layer_type].fields[field])
            except:
                field_values.append(None)

        field_values.append(len(packet))
        field_values.append(len(packet[layer_type].payload))
        field_values.append(packet[layer_type].payload.original)

        rows.append(field_values)

    df = pd.DataFrame(rows, columns=dataframe_fields)
    return df

In [32]:
from tqdm.notebook import tqdm
import pandas as pd


dfs = []

# Outer tqdm loop for processing packets
for packets, label, experiment_attack_directory in tqdm(all_packets, desc="Processing Packets"):
    # Removed print statements for cleaner output

    session_extractor = get_sessions_with_timeout(TIMEOUT, experiment_attack_directory)
    session_dict = packets.sessions(session_extractor)
    # Removed print statements for cleaner output

    session_dfs = dict()
    
    # Inner tqdm loop for processing sessions
    inner_tqdm = tqdm(session_dict.items(), desc="Processing Sessions", leave=False)
    for flow_key, flow in inner_tqdm:
        # Update the description with the current flow size
        inner_tqdm.set_description(f"Processing flow size: {len(flow)}")
        session_dfs[flow_key] = build_dataframe(flow)
        # break
        
    flow_df = pd.concat(session_dfs)
    flow_df['label'] = label
    dfs.append(flow_df)


Processing Packets:   0%|          | 0/41 [00:00<?, ?it/s]

Processing Sessions:   0%|          | 0/16 [00:00<?, ?it/s]

Processing Sessions:   0%|          | 0/7 [00:00<?, ?it/s]

Processing Sessions:   0%|          | 0/110 [00:00<?, ?it/s]

Processing Sessions:   0%|          | 0/132 [00:00<?, ?it/s]

...

Processing Sessions:   0%|          | 0/577 [00:00<?, ?it/s]

Processing Sessions:   0%|          | 0/7 [00:00<?, ?it/s]

Processing Sessions:   0%|          | 0/534 [00:00<?, ?it/s]

Processing Sessions:   0%|          | 0/526 [00:00<?, ?it/s]

Processing Sessions:   0%|          | 0/771 [00:00<?, ?it/s]

In [33]:
df = pd.concat(dfs)

In [34]:
df.iloc[0]

ip_version                             4
ip_ihl                                 5
ip_tos                                 0
ip_len                                46
ip_id                               2437
ip_flags                              DF
ip_frag                                0
ip_ttl                               128
ip_proto                               6
ip_chksum                              0
ip_src                      192.168.1.20
ip_dst                      192.168.1.21
ip_options                             0
time                   1587763927.678300
tcp_sport                          50045
tcp_dport                           2404
tcp_seq                       2931287365
tcp_ack                       2338122368
tcp_dataofs                            5
tcp_reserved                           0
tcp_flags                             PA
tcp_window                         16425
tcp_chksum                         33690
tcp_urgptr                             0
tcp_options     

In [35]:
df.columns

Index(['ip_version', 'ip_ihl', 'ip_tos', 'ip_len', 'ip_id', 'ip_flags',
       'ip_frag', 'ip_ttl', 'ip_proto', 'ip_chksum', 'ip_src', 'ip_dst',
       'ip_options', 'time', 'tcp_sport', 'tcp_dport', 'tcp_seq', 'tcp_ack',
       'tcp_dataofs', 'tcp_reserved', 'tcp_flags', 'tcp_window', 'tcp_chksum',
       'tcp_urgptr', 'tcp_options', 'size', 'payload', 'payload_raw', 'label'],
      dtype='object')

In [36]:
df['ip_flags']

[1, '192.168.1.20', '192.168.1.21', 2404, 50045, 'TCP', 'm_sp_na_1_DoS']  0    DF
                                                                          1    DF
                                                                          2    DF
                                                                          3    DF
                                                                          4    DF
                                                                               ..
[1, '192.168.1.22', '192.168.1.29', 2404, 39973, 'TCP', 'c_sc_na_1_DoS']  4    DF
                                                                          5    DF
                                                                          6    DF
                                                                          7    DF
                                                                          8    DF
Name: ip_flags, Length: 518377, dtype: object

In [37]:
df.index[0]

("[1, '192.168.1.20', '192.168.1.21', 2404, 50045, 'TCP', 'm_sp_na_1_DoS']", 0)

In [38]:
df.index.names = ['flow', 'packet_number']

In [39]:
df.groupby(level='flow')['ip_id'].count().mean(),df.groupby(level='flow')['ip_id'].count().median()

(38.24813694385007, 8.0)

In [40]:
df['ip_flags']=df['ip_flags'].map(lambda x: str(x))
df['tcp_flags']=df['tcp_flags'].map(lambda x: str(x))

In [4]:
df.to_parquet(f"data/iec104_{TIMEOUT}.parquet")

In [42]:
import scipy.stats
scipy.stats.mstats.mquantiles(df.groupby(level='flow')['ip_id'].count(),prob=[0.25,0.5,0.75,0.9,0.99])

array([ 8.,  8.,  9., 10., 26.])

# Split

In [33]:
df = pd.read_parquet(f"data/iec104_{TIMEOUT}.parquet")
df.head(1)

,,ip_version,ip_ihl,ip_tos,ip_len,ip_id,ip_flags,ip_frag,ip_ttl,ip_proto,ip_chksum,...,tcp_reserved,tcp_flags,tcp_window,tcp_chksum,tcp_urgptr,tcp_options,size,payload,payload_raw,label
flow,packet_number,,,,,,,,,,,,,,,,,,,,,
"[1, '192.168.1.20', '192.168.1.21', 2404, 50045, 'TCP', 'm_sp_na_1_DoS']",0,4,5,0,46,2437,DF,0,128,6,0,...,0,PA,16425,33690,0,0,60,6,b'h\x04\x07\x00\x00\x00',m_sp_na_1_DoS


In [31]:
from feature_engine.encoding import OrdinalEncoder
od = OrdinalEncoder(encoding_method='arbitrary')
od.fit(df[['label']])
df[['label']]= od.transform(df[['label']])
tmp = pd.DataFrame(df['label'].unique().reshape(-1, 1),columns=['label'])
pd.concat((od.inverse_transform(tmp),tmp),axis=1)
df = df.reset_index('packet_number')

In [32]:
import random
from constants import *

def generate_dataset(df,number_packets):
    if isinstance(number_packets,int):
        # df = df.groupby(level='flow').filter(lambda x: len(x) <= number_packets)
        df = df[df['packet_number'].isin(list(range(number_packets)))]
        df = df.groupby(level='flow').filter(lambda x: len(x) <= number_packets)
    else:
        pass
    df['time']=df['time'].astype(float)

    TRAIN_RATE= 0.8
   
    indexes = list(set(df.index))
    random.Random(1).shuffle(indexes)
    TRAIN_SIZE = int(len(indexes)*TRAIN_RATE)
    TEST_SIZE = len(indexes)-TRAIN_SIZE
    TRAIN_SAMPLES = indexes[:TRAIN_SIZE]
    TEST_SAMPLES = indexes[:TEST_SIZE]
    df_train_initial = df.loc[TRAIN_SAMPLES]
    df_test = df.loc[TEST_SAMPLES]
    
    indexes = list(set(df_train_initial.index))
    random.Random(1).shuffle(indexes)
    TRAIN_SIZE = int(len(indexes)*TRAIN_RATE)
    TEST_SIZE = len(indexes)-TRAIN_SIZE
    TRAIN_SAMPLES = indexes[:TRAIN_SIZE]
    TEST_SAMPLES = indexes[:TEST_SIZE]
    df_train = df_train_initial.loc[TRAIN_SAMPLES]
    df_validation = df_train_initial.loc[TEST_SAMPLES]
    
    print(df_train.shape, df_validation.shape, df_test.shape)
    # len(set(df_train.index))
    # display(df_train['label'].value_counts())
    # display(df_validation['label'].value_counts())
    # display(df_test['label'].value_counts())
    
    print('TRAIN',df_train.index.nunique(),df_validation.index.nunique(),df_test.index.nunique())# df_train.index.nunique(),df_validation.index.nunique(),df_test.index.nunique()
    df_train.to_parquet(f'data/iec104_train_{TIMEOUT}_{number_packets}.parquet')
    df_validation.to_parquet(f'data/iec104_validation_{TIMEOUT}_{number_packets}.parquet')
    df_test.to_parquet(f'data/iec104_test_{TIMEOUT}_{number_packets}.parquet')
    
for number_packets in NUMBER_PACKETS_OPTIONS:
    generate_dataset(df,number_packets)

(17346, 30) (4338, 30) (5422, 30)
TRAIN 8673 2169 2711
(52012, 30) (12999, 30) (16261, 30)
TRAIN 8673 2169 2711
(72827, 30) (18188, 30) (22752, 30)
TRAIN 8673 2169 2711
(75081, 30) (18736, 30) (23264, 30)
TRAIN 8673 2169 2711
(298295, 30) (72810, 30) (58165, 30)
TRAIN 8673 2169 2711
